In [ ]:
from google.colab import files
uploaded = files.upload()


Saving breast+cancer.zip to breast+cancer.zip
Saving Iris.csv to Iris.csv


In [ ]:
import csv, math, random
from collections import Counter, defaultdict

RANDOM_SEED = 42
random.seed(RANDOM_SEED)

In [ ]:
def load_iris_csv(path="Iris.csv"):
    X, y = [], []
    with open(path, "r", newline="") as f:
        rdr = csv.DictReader(f)
        for row in rdr:
            X.append([
                float(row["SepalLengthCm"]),
                float(row["SepalWidthCm"]),
                float(row["PetalLengthCm"]),
                float(row["PetalWidthCm"]),
            ])
            y.append(row["Species"])
    return X, y

In [ ]:
def train_val_test_split(X, y, ratios=(0.8, 0.1, 0.1), seed=RANDOM_SEED, stratify=True):
    assert abs(sum(ratios)-1.0) < 1e-9
    n = len(X)
    idxs = list(range(n))
    if stratify:
        # group by class
        by_cls = defaultdict(list)
        for i, label in enumerate(y):
            by_cls[label].append(i)
        train, val, test = [], [], []
        for cls, inds in by_cls.items():
            random.Random(seed).shuffle(inds)
            n_c = len(inds)
            n_train = int(ratios[0]*n_c)
            n_val   = int(ratios[1]*n_c)
            train += inds[:n_train]
            val   += inds[n_train:n_train+n_val]
            test  += inds[n_train+n_val:]
    else:
        random.Random(seed).shuffle(idxs)
        n_train = int(ratios[0]*n)
        n_val   = int(ratios[1]*n)
        train, val, test = idxs[:n_train], idxs[n_train:n_train+n_val], idxs[n_train+n_val:]

    def take(idxs):
        return [X[i] for i in idxs], [y[i] for i in idxs]
    return take(train), take(val), take(test)


In [ ]:
# CART Decision Tree (Gini)

class TreeNode:
    __slots__ = ("is_leaf","prediction","feature","threshold","left","right")
    def __init__(self, is_leaf, prediction=None, feature=None, threshold=None, left=None, right=None):
        self.is_leaf = is_leaf
        self.prediction = prediction
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right

def gini_impurity(labels):
    total = len(labels)
    if total == 0: return 0.0
    counts = Counter(labels)
    return 1.0 - sum((c/total)**2 for c in counts.values())

def best_split_numeric(X, y):
    n = len(X)
    if n <= 1:
        return None, None, None
    m = len(X[0])
    base_imp = gini_impurity(y)
    best_gain = 0.0
    best_f, best_thr, best_partition = None, None, None

    for f in range(m):
        # sort by feature f
        pairs = sorted(zip(X, y), key=lambda t: t[0][f])
        xs = [p[0][f] for p in pairs]
        ys = [p[1] for p in pairs]
        # candidate thresholds: midpoints between distinct consecutive values
        left_counts = Counter()
        right_counts = Counter(ys)
        left_n = 0
        right_n = n
        for i in range(n-1):
            cls = ys[i]
            left_counts[cls] += 1; left_n += 1
            right_counts[cls] -= 1; right_n -= 1
            if xs[i] == xs[i+1]:
                continue
            thr = 0.5*(xs[i] + xs[i+1])

            # compute gini for left/right
            gl = 1.0 - sum((c/left_n)**2 for c in left_counts.values())
            gr = 1.0 - sum((c/right_n)**2 for c in right_counts.values())
            weighted = (left_n/n)*gl + (right_n/n)*gr
            gain = base_imp - weighted
            if gain > best_gain:
                best_gain = gain
                best_f = f
                best_thr = thr

    if best_f is None:
        return None, None, None

    # Partition using best split
    left_idx = [i for i, x in enumerate(X) if x[best_f] <= best_thr]
    right_idx = [i for i, x in enumerate(X) if x[best_f] > best_thr]
    return best_f, best_thr, (left_idx, right_idx)

class DecisionTree:
    def __init__(self, max_depth=10, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None

    def fit(self, X, y):
        self.root = self._grow(X, y, depth=0)

    def _leaf(self, y):
        # majority class
        return TreeNode(is_leaf=True, prediction=Counter(y).most_common(1)[0][0])

    def _grow(self, X, y, depth):
        if depth >= self.max_depth or len(X) < self.min_samples_split or gini_impurity(y) == 0.0:
            return self._leaf(y)
        f, thr, parts = best_split_numeric(X, y)
        if f is None or parts is None:
            return self._leaf(y)
        left_idx, right_idx = parts
        if len(left_idx) == 0 or len(right_idx) == 0:
            return self._leaf(y)
        Xl = [X[i] for i in left_idx]; yl = [y[i] for i in left_idx]
        Xr = [X[i] for i in right_idx]; yr = [y[i] for i in right_idx]
        left = self._grow(Xl, yl, depth+1)
        right = self._grow(Xr, yr, depth+1)
        node = TreeNode(is_leaf=False, feature=f, threshold=thr, left=left, right=right)
        return node

    def _predict_one(self, x):
        node = self.root
        while not node.is_leaf:
            if x[node.feature] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node.prediction

    def predict(self, X):
        return [self._predict_one(x) for x in X]


In [ ]:
#Bagging

class BaggingClassifier:
    def __init__(self, base_params=None, n_estimators=50, max_samples=None, random_state=RANDOM_SEED):
        self.base_params = base_params or {}
        self.n_estimators = n_estimators
        self.max_samples = max_samples  # None -> len(train)
        self.random_state = random_state
        self.trees = []

    def fit(self, X, y):
        rng = random.Random(self.random_state)
        n = len(X)
        m = self.max_samples or n
        self.trees = []
        for t in range(self.n_estimators):
            idxs = [rng.randrange(n) for _ in range(m)]  # bootstrap with replacement
            Xb = [X[i] for i in idxs]
            yb = [y[i] for i in idxs]
            tree = DecisionTree(**self.base_params)
            tree.fit(Xb, yb)
            self.trees.append(tree)

    def predict(self, X):
        # majority vote
        votes = []
        for x in X:
            preds = [tree._predict_one(x) for tree in self.trees]
            votes.append(Counter(preds).most_common(1)[0][0])
        return votes

def accuracy(y_true, y_pred):
    return sum(yt==yp for yt, yp in zip(y_true, y_pred))/len(y_true)


In [ ]:
# Run on Iris

X, y = load_iris_csv("Iris.csv")
(trainX, trainY), (valX, valY), (testX, testY) = train_val_test_split(X, y, ratios=(0.7,0.15,0.15), seed=RANDOM_SEED, stratify=True)

bag = BaggingClassifier(
    base_params={"max_depth":5, "min_samples_split":2},
    n_estimators=50,
    max_samples=None,
    random_state=RANDOM_SEED
)
bag.fit(trainX, trainY)

val_pred = bag.predict(valX)
test_pred = bag.predict(testX)
print(f"[Iris] Validation accuracy: {accuracy(valY, val_pred):.4f}")
print(f"[Iris] Test accuracy:       {accuracy(testY, test_pred):.4f}")

[Iris] Validation accuracy: 0.9048
[Iris] Test accuracy:       0.9583


Breast Cancer

In [ ]:
import csv, math, random, zipfile, os
from collections import Counter, defaultdict

RANDOM_SEED = 42
random.seed(RANDOM_SEED)

In [ ]:
# Prepare the data
if os.path.exists("breast+cancer.zip"):
    with zipfile.ZipFile("breast+cancer.zip","r") as z:
        z.extractall("breast_cancer_ucI")  # folder
data_path = "breast_cancer_ucI/breast-cancer.data"
assert os.path.exists(data_path), "breast-cancer.data not found. Ensure breast+cancer.zip is uploaded."

def load_breast_cancer_original(path=data_path):
    """
    Returns X as list of lists of strings (categorical, with 'unknown' instead of '?'),
    y as list in {-1, +1}, mapping:
      'no-recurrence-events' -> +1
      'recurrence-events'    -> -1
    """
    X, y = [], []
    with open(path, "r", newline="") as f:
        rdr = csv.reader(f)
        for row in rdr:
            if not row:
                continue
            # columns per UCI: class, age, menopause, tumor-size, inv-nodes, node-caps, deg-malig, breast, breast-quad, irradiat
            cls = row[0].strip()
            feats = [c.strip() for c in row[1:]]
            feats = ["unknown" if v=="?" else v for v in feats]
            # robust: deg-malig is numeric-ish; still treat as categorical string
            if cls == "no-recurrence-events":
                y.append(+1)
            elif cls == "recurrence-events":
                y.append(-1)
            else:
                # skip if unexpected label
                continue
            X.append(feats)
    return X, y

Xc, yc = load_breast_cancer_original(data_path)

def stratified_split(X, y, ratios=(0.8,0.1,0.1), seed=RANDOM_SEED):
    by_cls = defaultdict(list)
    for i, label in enumerate(y):
        by_cls[label].append(i)
    train, val, test = [], [], []
    for cls, inds in by_cls.items():
        r = random.Random(seed)
        r.shuffle(inds)
        n = len(inds)
        n_train = int(ratios[0]*n)
        n_val   = int(ratios[1]*n)
        train += inds[:n_train]
        val   += inds[n_train:n_train+n_val]
        test  += inds[n_train+n_val:]
    def take(idxs):
        return [X[i] for i in idxs], [y[i] for i in idxs]
    return take(train), take(val), take(test)

(trainX, trainY), (valX, valY), (testX, testY) = stratified_split(Xc, yc, ratios=(0.8,0.1,0.1), seed=RANDOM_SEED)


In [ ]:
# Decision Stump (categorical)

class CatStump:
    """
    h(x) = s * ( 1 if x[f] == cat else -1 ), with s in {+1, -1}
    """
    def __init__(self):
        self.feature = None
        self.category = None
        self.sign = +1  # orientation

    def fit_weighted(self, X, y, w):
        n = len(X); m = len(X[0])
        best_err = 1e9
        best = (None, None, +1)
        # Precompute categories per feature
        for f in range(m):
            # collect all categories for this feature
            cats = set(x[f] for x in X)
            for cat in cats:
                # h_pos: predict +1 if equals cat, else -1
                err_pos = 0.0
                for i in range(n):
                    pred = +1 if X[i][f] == cat else -1
                    if pred != y[i]:
                        err_pos += w[i]
                # h_neg: flip orientation (predict -1 if equals cat, else +1)
                err_neg = 0.0
                for i in range(n):
                    pred = -1 if X[i][f] == cat else +1
                    if pred != y[i]:
                        err_neg += w[i]
                if err_pos < best_err:
                    best_err = err_pos
                    best = (f, cat, +1)
                if err_neg < best_err:
                    best_err = err_neg
                    best = (f, cat, -1)

        self.feature, self.category, self.sign = best

        # return weighted error found
        return best_err

    def predict_one(self, x):
        hit = (x[self.feature] == self.category)
        raw = +1 if hit else -1
        return self.sign * raw

    def predict(self, X):
        return [self.predict_one(x) for x in X]


In [ ]:
#AdaBoost (SAMME for binary with +/-1 labels)

class AdaBoost:
    def __init__(self, T=100):
        self.T = T
        self.learners = []
        self.alphas = []

    def fit(self, X, y):
        n = len(X)
        w = [1.0/n]*n
        self.learners = []
        self.alphas = []
        eps = 1e-12
        for t in range(self.T):
            stump = CatStump()
            err = stump.fit_weighted(X, y, w)
            err = min(max(err, eps), 1.0 - eps)  # clamp
            alpha = 0.5*math.log((1.0 - err)/err)
            # update weights
            for i in range(n):
                pred = stump.predict_one(X[i])
                w[i] = w[i] * math.exp(-alpha * y[i] * pred)
            # normalize
            Z = sum(w)
            w = [wi/Z for wi in w]
            self.learners.append(stump)
            self.alphas.append(alpha)

    def predict_scores(self, X):
        # returns real-valued margin sum(alpha_t * h_t(x))
        scores = [0.0]*len(X)
        for alpha, h in zip(self.alphas, self.learners):
            preds = h.predict(X)
            for i, p in enumerate(preds):
                scores[i] += alpha * p
        return scores

    def predict(self, X):
        scores = self.predict_scores(X)
        # sign -> {+1, -1}
        return [ +1 if s >= 0 else -1 for s in scores ]

def accuracy(y_true, y_pred):
    return sum(yt==yp for yt, yp in zip(y_true, y_pred))/len(y_true)

def to_label_name_binary(y_pm1):
    # map back to strings for readability (optional)
    return ["no-recurrence-events" if v==+1 else "recurrence-events" for v in y_pm1]


In [ ]:
# --------- Train / Evaluate ---------

ada = AdaBoost(T=150)
ada.fit(trainX, trainY)

val_pred = ada.predict(valX)
test_pred = ada.predict(testX)

print(f"[Breast-Cancer] Validation accuracy: {accuracy(valY, val_pred):.4f}")
print(f"[Breast-Cancer] Test accuracy:       {accuracy(testY, test_pred):.4f}")

[Breast-Cancer] Validation accuracy: 0.6786
[Breast-Cancer] Test accuracy:       0.7667


In [ ]:
# simple confusion matrix on test
def confusion_matrix_binary(y_true, y_pred):
    # +1 = no-recurrence, -1 = recurrence
    tp = sum(1 for yt, yp in zip(y_true, y_pred) if yt==+1 and yp==+1)
    tn = sum(1 for yt, yp in zip(y_true, y_pred) if yt==-1 and yp==-1)
    fp = sum(1 for yt, yp in zip(y_true, y_pred) if yt==-1 and yp==+1)
    fn = sum(1 for yt, yp in zip(y_true, y_pred) if yt==+1 and yp==-1)
    return {"TP(+1)":tp, "TN(-1)":tn, "FP":fp, "FN":fn}

print("[Breast-Cancer] Test Confusion Matrix:", confusion_matrix_binary(testY, test_pred))

[Breast-Cancer] Test Confusion Matrix: {'TP(+1)': 20, 'TN(-1)': 3, 'FP': 6, 'FN': 1}
